In [ ]:
%%bash

pip install --upgrade pip
pip install  --disable-pip-version-check \
  torch==1.13.1 \
  torchdata==0.5.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 85.9 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torchdata==0.7.0, but you have torchdata 0.5.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.


In [ ]:
!pip install transformers==4.27.2 --quiet
!pip install datasets==2.11.0 --quiet
!pip install evaluate==0.4.0 --quiet
!pip install rouge_score==0.1.2 --quiet
!pip install loralib==0.1.1 --quiet
!pip install peft==0.3.0 --quiet

# !pip install transformers==4.27.2 \
# !pip install datasets>=2.11.0 \
# !pip install evaluate>=0.4.0 \
# !pip install rouge_score==0.1.2 \
# !pip install loralib==0.1.1 \
# !pip install peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

- DialogSum Hugging Face dataset
  - 10000+ dialogue with summary

In [ ]:
huggingface_dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(huggingface_dataset_name)
dataset

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [ ]:
model_name='google/flan-t5-small'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

find the trainable parameters in the model

In [ ]:
def print_numer_of_trainable_model_parameters(model):
  trainable_model_parameters = 0
  all_model_parameters = 0
  for _, param in model.named_parameters():
    all_model_parameters += param.numel()
    if param.requires_grad:
      trainable_model_parameters += param.numel()

  return f'trainable moel parameters: {trainable_model_parameters}\nall model parameters: {all_model_parameters}\npercentage of trainable model parameters: {trainable_model_parameters/all_model_parameters}'

print(print_numer_of_trainable_model_parameters(original_model))

trainable moel parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 1.0


## Test the model with Zero Shot Inferencing

In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarisze the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs['input_ids'],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '_'.join(' ' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
INPUT PROMPT:

Summarisze the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That

# Perform Full Fine-Tuning

## Process the Dialog-Summary Dataset


In [ ]:
def tokenize_function(example):
  start_prompt = 'Summarize the following conversation.\n\n'
  end_prompt = '\n\nSummary: '
  prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
  example['input_ids'] = tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').input_ids
  example['labels'] = tokenizer(example['summary'], padding='max_length', truncation=True, return_tensors='pt').input_ids

  return example

# The datast acually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

- filter down the dataset to make the training faster

In [ ]:
# tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
print(f'Shape of the datasets:')
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

Shape of the datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)


## Fine tune the model with the preprocessed Dataset
- Hugging face Trainer Class
- pass Model and Dataset to Trainer Class

In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    learning_rate=1e-5,
    num_train_epochs=50,
    weight_decay=0.01,
    # max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

# train the data
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,48.581000
1000,42.730000
1500,42.041500
2000,41.921500
2500,41.898000
3000,41.866500
3500,41.823000
4000,41.821000
4500,41.890500
5000,41.870000


TrainOutput(global_step=15580, training_loss=42.095651476251604, metrics={'train_runtime': 14144.2984, 'train_samples_per_second': 8.809, 'train_steps_per_second': 1.102, 'total_flos': 2.31619568861184e+16, 'train_loss': 42.095651476251604, 'epoch': 10.0})

- find the size of the model trained

In [ ]:
! ls -alh $output_dir

total 12K
drwxr-xr-x 3 root root 4.0K Jan  6 16:08 .
drwxr-xr-x 1 root root 4.0K Jan  6 16:08 ..
drwxr-xr-x 3 root root 4.0K Jan  6 16:08 runs


## Evaluate the Model Qualitatively (Human Evaluation)

In [ ]:
# instruct_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir, torch_dtype=torch.bfloat16)
instruct_model = trainer.model.to('cpu')

## 2.4 - Evaluate the Model Quantitatively (with ROUGE Metric)

In [ ]:
from tqdm import tqdm
# dialogues = dataset['test'][0:10]['dialogue']
# human_baseline_summaries = dataset['test'][0:10]['summary']

dialogues = dataset['test']['dialogue']
human_baseline_summaries = dataset['test']['summary']

original_model_summaries = []
trained_model_summaries = []

pbar = tqdm(total=len(dialogues))
for _, dialogue in enumerate(dialogues):
  prompt = f"""
SUmmarize the following conversation.
{dialogue}
Summary: """

  input_ids = tokenizer(prompt, return_tensors='pt').input_ids
  original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
  original_model_summaries.append(original_model_text_output)

  trained_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
  trained_model_text_output = tokenizer.decode(trained_model_outputs[0], skip_special_tokens=True)
  trained_model_summaries.append(trained_model_text_output)
  pbar.update(1)
pbar.close()

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, trained_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['huam_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (1030 > 512). Running this sequence through the model will result in indexing errors


,huam_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,"I'm sorry, but I've got to the point of not le...",#Person1#: I need to take a dictation for all ...
1,In order to prevent employees from wasting tim...,SUmmarize the conversation.,This memo should be sent to all employees befo...
2,Ms. Dawson takes a dictation for #Person1# abo...,"#Person1#: I'm sorry, but I'm not sure that yo...",Copyright 2001-2002 2001-2002 2001-2002 2...
3,#Person2# arrives late because of traffic jam....,The traffic jam is a lot more stressful than d...,SUMITTED_BY SUMITTED_BY SUMITTED_BY SUMITTED B...
4,#Person2# decides to follow #Person1#'s sugges...,Taking the subway would be a better option.,SUmmarmarancing with a car.


ROUGE metrics
- orginal model with human base line
- full fine tuned with human base line

In [ ]:
rouge = evaluate.load('rouge')

test = df.loc[2:4, :]
original_model_summaries = test['original_model_summaries'].to_list()
human_baseline_summaries = test['huam_baseline_summaries'].to_list()
trained_model_summaries = test['instruct_model_summaries'].to_list()

original_model_result = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True
)

instruction_model_result = rouge.compute(
    predictions=trained_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True
)

print('Original Model')
print(original_model_result)
print('Instruction Tuned')
print(instruction_model_result)

Original Model
{'rouge1': 0.1101190476190476, 'rouge2': 0.022222222222222223, 'rougeL': 0.07341269841269842, 'rougeLsum': 0.07341269841269842}
Instruction Tuned
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


# 3 Parameter Efficient Fine-Tuning (PEFT)

## LoRA fine-tuning

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=['q', 'v'],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM #FLAN-T5
)

# Add LoRA adapter layers/parametes to the original LLM to be trained
peft_model = get_peft_model(
    original_model,
    lora_config
    )

print(print_numer_of_trainable_model_parameters(peft_model))

trainable moel parameters: 1376256
all model parameters: 78337408
percentage of trainable model parameters: 0.017568311680672407


## Train PEFT Adapter

In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning
    num_train_epochs=10,
    logging_steps=100,
    # max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args = peft_training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

peft_trainer.train()

# save model and tokenizer
peft_model_path = './peft-dialogue-summary-checkpoint-local'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


## inject peft adapter to a model

In [ ]:
# from peft import PeftModel, PEFT_TYPE_TO_CONFIG_MAPPING

# pedft_model_base = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-small', torch_dtype=bfloat16)
# tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-small')

# peft_model = PeftModel.from_pretrained(
#     peft_model_base,
#     './model_check_point',
#     torch_dtype=torch.bfloat16,
#     is_trainable=False # save memore for gradient
#     )

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

['#Person2# arrives late because of traffic jam. #Person1# persuades #Person2# to use public transportations to keep healthy and to protect the environment.',
 "#Person2# decides to follow #Person1#'s suggestions on quitting driving to work and will try to use public transportations."]

In [ ]:
rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries
)

{'rouge1': 0.1290322580645161,
 'rouge2': 0.034482758620689655,
 'rougeL': 0.0967741935483871,
 'rougeLsum': 0.0967741935483871}